In [1]:
!pip install https://gitlab.com/trungtv/vi_spacy/-/raw/master/packages/vi_core_news_lg-3.6.0/dist/vi_core_news_lg-3.6.0.tar.gz -q
!pip install pyvi qdrant-client sentence-transformers -q
!python -m spacy validate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 MB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.4/917.4 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 109.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 2

In [2]:
!python -m spacy validate

2025-06-18 12:44:32.942760: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750250672.965956      86 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750250672.972961      86 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.6.1) =================
ℹ spaCy installation: /usr/local/lib/python3.11/dist-packages/spacy

NAME              SPACY            VERSION                            
vi_core_news_lg   >=3.6.0,<3.7.0   3.6.0   ✔
en_core_web_sm    >=3.8.0,<3.9.0   3.8.0   --> 3.6.0     


============================== Install updates ==============================
Use t

In [3]:
import re
import pandas as pd
import json
import pickle
import difflib
import spacy
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm # Sử dụng tqdm cho notebook
import os

# ==============================================================================
# === 1. CẤU HÌNH CÁC MÔ HÌNH VÀ ĐƯỜNG DẪN =======================================
# ==============================================================================

# --- Danh sách các model bạn muốn sử dụng để tạo embeddings ---
# Tên (name) sẽ được dùng để đặt tên cho file output.
MODEL_CONFIGURATIONS = [
    {
        "name": "halong_finetuned",
        "model_path": "/kaggle/input/embedding-train/model_test"
    },
    {
        "name": "phobert_finetuned",
        "model_path": "/kaggle/input/embedding-train-sup-simcse-vietnamese-phobert-base/sup_SimCSE_VietNamese_phobert_base"
    },
    {
        "name": "halong_base",
        "model_path": "hiieu/halong_embedding"
    },
    {
        "name": "phobert_base",
        "model_path": "VoVanPhuc/sup-SimCSE-VietNamese-phobert-base"
    }
]

# --- Đường dẫn file input và thư mục output ---
INPUT_CSV_PATH = "/kaggle/input/data-uit/train.csv"
OUTPUT_DIR = "/kaggle/working/processed_data" # Thư mục để lưu tất cả các file output

# Tạo thư mục output nếu chưa tồn tại
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Các file output sẽ được lưu tại: {OUTPUT_DIR}")


# ==============================================================================
# === 2. KHỞI TẠO CÁC CÔNG CỤ VÀ HÀM HỖ TRỢ =====================================
# ==============================================================================

# --- Khởi tạo spaCy NLP model ---
NLP_PROCESSOR = None
try:
    NLP_PROCESSOR = spacy.load("vi_core_news_lg")
    print("✔ Đã tải xong mô hình spaCy 'vi_core_news_lg'.")
except OSError:
    print("❌ Lỗi: Không tìm thấy mô hình spaCy 'vi_core_news_lg'.")
    print("  Vui lòng tải mô hình này về bằng lệnh: python -m spacy download vi_core_news_lg")
    print("  Chương trình sẽ dừng lại vì không thể chia chunk.")
    # exit() # Bỏ comment nếu muốn dừng hẳn chương trình

# --- Các hàm xử lý văn bản ---
def remove_redundant_repeats(text):
    """Xoá các đoạn bị lặp như "Điều 9. Điều 9."""
    return re.sub(r"(Điều\s+\d+\.)\s+\1", r"\1", text)

def split_into_semantic_chunks(text, nlp_processor, max_words_per_chunk=200):
    """Chia văn bản thành các chunk ngữ nghĩa sử dụng spaCy."""
    if nlp_processor is None:
        return [text] if text.strip() else []

    text = remove_redundant_repeats(text)
    doc = nlp_processor(text)
    
    sentences = [sent.text.strip() for sent in doc.sents if len(sent.text.strip()) > 10 and not re.fullmatch(r'[-–]+', sent.text.strip())]
    
    if not sentences:
        return []

    # Gộp các câu lại thành các chunk lớn hơn (khoảng 200 từ)
    processed_chunks = []
    current_chunk_buffer = ""
    for sent in sentences:
        if len((current_chunk_buffer + " " + sent).split()) > max_words_per_chunk and current_chunk_buffer:
            processed_chunks.append(current_chunk_buffer.strip())
            current_chunk_buffer = sent
        else:
            if not current_chunk_buffer:
                current_chunk_buffer = sent
            else:
                current_chunk_buffer += " " + sent
    
    if current_chunk_buffer:
        processed_chunks.append(current_chunk_buffer.strip())
        
    return processed_chunks

def clean_text(text):
    """Dọn dẹp văn bản cơ bản."""
    cleaned_text = re.sub(r'[^\w\s.,]', '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

def strings_similar(s1, s2, threshold=0.95):
    """Kiểm tra sự giống nhau của hai chuỗi."""
    return difflib.SequenceMatcher(None, s1, s2).ratio() > threshold

# ==============================================================================
# === 3. CHUNKING VÀ XỬ LÝ DỮ LIỆU (CHẠY 1 LẦN) ================================
# ==============================================================================

print("\nBắt đầu giai đoạn 1: Chunking và xử lý dữ liệu...")

try:
    df = pd.read_csv(INPUT_CSV_PATH)
    print(f"✔ Đã đọc thành công file: {INPUT_CSV_PATH} ({len(df)} hàng)")
except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy file {INPUT_CSV_PATH}. Vui lòng kiểm tra đường dẫn.")
    df = pd.DataFrame()

all_payloads = []
unique_chunks_text = [] # Chỉ lưu trữ các đoạn text của chunk duy nhất
test_data = []
global_id_counter = 0

if not df.empty and NLP_PROCESSOR:
    # Sử dụng tqdm để theo dõi tiến trình
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Đang xử lý các context"):
        original_text_context = str(row.get('context', ''))
        
        # Bỏ qua các context rỗng
        if not original_text_context.strip():
            continue

        document_title = str(row.get("document", ""))
        article_info = str(row.get("article", ""))
        question = str(row.get("question", ""))

        # Chia context thành các chunk
        document_chunks = split_into_semantic_chunks(original_text_context, NLP_PROCESSOR)

        chunk_ids_for_this_question = []

        for chunk_text in document_chunks:
            cleaned_chunk = clean_text(chunk_text)
            if not cleaned_chunk:
                continue

            # Kiểm tra trùng lặp
            is_duplicate = any(strings_similar(cleaned_chunk, kept_chunk) for kept_chunk in unique_chunks_text)

            if not is_duplicate:
                unique_chunks_text.append(cleaned_chunk)
                
                payload = {
                    "id": global_id_counter,
                    "title": document_title,
                    "article": clean_text(article_info),
                    "context": cleaned_chunk,
                    "passage": cleaned_chunk
                }
                all_payloads.append(payload)
                chunk_ids_for_this_question.append(global_id_counter)
                global_id_counter += 1

        if question and chunk_ids_for_this_question:
            test_data.append({"question": clean_text(question), "label": chunk_ids_for_this_question})

    # --- Lưu file JSONL chứa các chunk (payloads) ---
    output_jsonl_path = os.path.join(OUTPUT_DIR, "processed_data.jsonl")
    with open(output_jsonl_path, 'w', encoding='utf-8') as f:
        for payload_item in all_payloads:
            f.write(json.dumps(payload_item, ensure_ascii=False) + '\n')
    print(f"✔ Đã lưu {len(all_payloads)} chunks vào file: {output_jsonl_path}")

    # --- Lưu file test.csv ---
    output_test_csv_path = os.path.join(OUTPUT_DIR, "test.csv")
    if test_data:
        test_df = pd.DataFrame(test_data)
        test_df.to_csv(output_test_csv_path, index=False, encoding='utf-8')
        print(f"✔ Đã tạo và lưu {len(test_df)} mẫu vào file: {output_test_csv_path}")
    else:
        print("🟡 Không có dữ liệu test để tạo file.")
else:
    print("❌ Không có dữ liệu hoặc NLP Processor chưa được khởi tạo. Bỏ qua giai đoạn xử lý.")


# ==============================================================================
# === 4. TẠO VÀ LƯU EMBEDDINGS (CHẠY CHO TỪNG MODEL) ===========================
# ==============================================================================

print("\nBắt đầu giai đoạn 2: Tạo và lưu embeddings cho từng mô hình...")

if unique_chunks_text:
    for config in MODEL_CONFIGURATIONS:
        model_name = config["name"]
        model_path = config["model_path"]
        output_pkl_path = os.path.join(OUTPUT_DIR, f"embeddings_{model_name}.pkl")
        
        print(f"\n--- Đang xử lý model: {model_name} ---")
        
        try:
            # Tải mô hình embedding
            print(f"  - Đang tải model từ: {model_path}...")
            embedding_model = SentenceTransformer(model_path)
            
            # Tạo embeddings cho tất cả các chunk duy nhất (batch processing)
            print(f"  - Đang tạo embeddings cho {len(unique_chunks_text)} chunks...")
            chunk_embeddings = embedding_model.encode(
                unique_chunks_text, 
                show_progress_bar=True,
                batch_size=32 # Điều chỉnh batch_size nếu cần thiết
            )
            
            # Lưu embeddings vào file .pkl
            print(f"  - Đang lưu embeddings vào: {output_pkl_path}...")
            with open(output_pkl_path, 'wb') as f:
                pickle.dump(chunk_embeddings, f)
            print(f"✔ Hoàn tất! Đã lưu {len(chunk_embeddings)} vectors.")

        except Exception as e:
            print(f"❌ Lỗi khi xử lý model {model_name}: {e}")
            continue # Bỏ qua và tiếp tục với model tiếp theo
else:
    print("🟡 Không có chunk duy nhất nào để tạo embeddings.")

print("\n✨✨✨ HOÀN TẤT TẤT CẢ CÁC GIAI ĐOẠN! ✨✨✨")

2025-06-18 12:44:42.817423: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750250682.840960      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750250682.848341      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Các file output sẽ được lưu tại: /kaggle/working/processed_data
✔ Đã tải xong mô hình spaCy 'vi_core_news_lg'.

Bắt đầu giai đoạn 1: Chunking và xử lý dữ liệu...
✔ Đã đọc thành công file: /kaggle/input/data-uit/train.csv (7806 hàng)


Đang xử lý các context:   0%|          | 0/7806 [00:00<?, ?it/s]

✔ Đã lưu 503 chunks vào file: /kaggle/working/processed_data/processed_data.jsonl
✔ Đã tạo và lưu 292 mẫu vào file: /kaggle/working/processed_data/test.csv

Bắt đầu giai đoạn 2: Tạo và lưu embeddings cho từng mô hình...

--- Đang xử lý model: halong_finetuned ---
  - Đang tải model từ: /kaggle/input/embedding-train/model_test...
  - Đang tạo embeddings cho 503 chunks...


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

...

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

  - Đang tạo embeddings cho 503 chunks...


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

  - Đang lưu embeddings vào: /kaggle/working/processed_data/embeddings_phobert_base.pkl...
✔ Hoàn tất! Đã lưu 503 vectors.

✨✨✨ HOÀN TẤT TẤT CẢ CÁC GIAI ĐOẠN! ✨✨✨
